In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('all')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

Using TensorFlow backend.


In [0]:
# zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/hate-speech-dataset-master.zip", 'r')
# zip_ref.extractall()
# zip_ref.close()
# file_path="/content/hate-speech-dataset-master/all_files"

In [0]:
# read_files=glob.glob(os.path.join(file_path,"*.txt"))
# data=[]
# file_name=[]
# for files in read_files:
#     with open(files,encoding='utf-8') as f:
#         data.append(f.read())
#         file_name.append(os.path.splitext(os.path.basename(files))[0])
# data=pd.DataFrame(data,index=file_name)
# data.head()

In [0]:
# Output=pd.read_csv("/content/hate-speech-dataset-master/annotations_metadata.csv",index_col=0)
# Output=Output.drop(['user_id','subforum_id','num_contexts'],axis=1).sort_index()
# Output.head()

In [0]:
# dataSet=pd.merge(data,Output,how='outer',left_index=True,right_index=True)
# dataSet=dataSet[(dataSet['label']=="noHate")|(dataSet['label']=='hate')]
# dataSet.rename(columns={0:'Texts'},inplace=True)
# dataSet['label']=(dataSet['label']=='hate').astype(int)
# dataSet.to_csv("hate-speech-dataset-master.csv")
# dataSet.head()

In [7]:
dataSet=pd.read_csv("/content/drive/My Drive/hate-speech-dataset-master.csv")
dataSet.set_index('Unnamed: 0',inplace=True)
dataSet.head()

Texts  label
Unnamed: 0                                                           
12834217_1   As of March 13th , 2014 , the booklet had been...      0
12834217_10  Thank you in advance. : ) Download the youtube...      0
12834217_2   In order to help increase the booklets downloa...      0
12834217_3   ( Simply copy and paste the following text int...      0
12834217_4   Click below for a FREE download of a colorfull...      1

In [8]:
train_data,test_data=sklearn.model_selection.train_test_split(dataSet,test_size=0.1)
print(train_data.shape)
print(train_data.head())

(9632, 2)
                                                        Texts  label
Unnamed: 0                                                          
13615314_1  To be honest .. i dont think this Muslim kille...      1
13617363_2                                              Susan      0
30698686_1                   I've seen those walls in Cuzco .      0
14028809_2  The OP has gone to sunnier climes for a couple...      0
30542596_2               But both Samis and Finns have N1C1 .      0


In [0]:
def create_tokens(lines):
  stop_words = set(nltk.corpus.stopwords.words('english'))
  words = set(nltk.corpus.words.words())
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  t_line=t_line.apply(lambda x: [w for w in x if w in words])
  t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  t_line= t_line.apply(lambda x: [word for word in x if word.isalpha()])
  return t_line

In [10]:
train_lines=create_tokens(train_data['Texts'])
print(train_lines)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(train_lines)
vocab=tokenizer.word_index
print(vocab)

Unnamed: 0
13615314_1    [honest, dont, think, thought, boy, like, raci...
13617363_2                                              [Susan]
30698686_1                                            [I, seen]
14028809_2       [The, gone, couple, I, close, explain, return]
30542596_2                                                   []
                                    ...                        
14301771_5                                                   []
30395750_1                        [The, Europa, Sir, A, Amazon]
30384268_2              [The, fed, go, order, put, notch, belt]
31483246_2                                       [transparency]
13589608_2                                              [trash]
Name: Texts, Length: 9632, dtype: object
{'i': 1, 'white': 2, 'like': 3, 'the': 4, 'people': 5, 'would': 6, 'one': 7, 'get': 8, 'see': 9, 'good': 10, 'know': 11, 'black': 12, 'time': 13, 'go': 14, 'school': 15, 'think': 16, 'want': 17, 'back': 18, 'say': 19, 'many': 20, 'got': 21, 'us

In [11]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
model=Word2Vec(train_lines,size=100,window=5,min_count=1,negative=5,workers=16)
print(model)

Word2Vec(vocab=6769, size=100, alpha=0.025)


In [12]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

6663

In [0]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded
train_x=encode_text(tokenizer,train_lines,200)

In [14]:
word_vectors=model.wv
print("Number of word vectors: {}".format(len(word_vectors.vocab)))
model.wv.most_similar_cosmul(positive=['sister', 'man'], negative=['brother'])

Number of word vectors: 6769


[('old', 1.0000141859054565),
 ('thing', 0.9999962449073792),
 ('black', 0.9999958276748657),
 ('together', 0.9999925494194031),
 ('thread', 0.9999897480010986),
 ('culture', 0.9999868273735046),
 ('country', 0.9999849200248718),
 ('next', 0.9999805092811584),
 ('least', 0.9999785423278809),
 ('link', 0.9999784827232361)]

In [15]:
MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 200
print(word_vectors['road'])

[ 7.23763779e-02 -8.42277482e-02 -2.05832273e-02 -3.01397722e-02
 -1.09372884e-01 -1.81684513e-02 -6.48421422e-02  1.71849431e-04
 -1.28041413e-02 -1.63034108e-02 -1.21507198e-01 -1.86223071e-02
 -8.21286999e-03  6.87308833e-02  8.88445899e-02  2.57651210e-02
  8.60247314e-02 -7.85426050e-03 -1.49087489e-01  3.55328768e-02
  2.48399675e-02  9.06785950e-03  4.51860018e-02 -1.77870300e-02
  1.22084692e-01  4.60070325e-03  1.06945038e-01  1.50077388e-01
 -4.49956320e-02 -5.96928485e-02 -9.56769288e-02 -9.19819772e-02
  1.58875450e-01  6.49809837e-02 -7.77385309e-02  4.97998819e-02
  1.03202298e-01 -1.41602028e-02 -1.28335625e-01  1.03549562e-01
  3.77154094e-04  1.59817711e-01 -1.62340075e-01  4.25937809e-02
  5.24495877e-02 -5.59755154e-02  1.39093688e-02 -9.13224444e-02
  2.85980757e-03 -2.61724740e-03 -1.11711606e-01 -5.62081672e-02
  2.50840299e-02  1.10012807e-01 -5.25329709e-02 -1.76928646e-03
 -9.21447948e-02 -1.03642203e-01  3.85509320e-02  4.83984686e-02
  8.96340758e-02  3.26360

In [0]:
# from keras.preprocessing.sequence import pad_sequences
# from collections import Counter
# Counter=Counter()
# word_index = {t[0]: i+1 for i,t in enumerate(vocab)}
# sequences = [[word_index.get(t, 0) for t in comment]
#              for comment in comments[:len(list_sentences_train)]]
# test_sequences = [[word_index.get(t, 0)  for t in comment] 
#                   for comment in comments[len(list_sentences_train):]]
# data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre", truncating="post")
# list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
# y = train_df[list_classes].values
# print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', y.shape)
# test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre",truncating="post")
# print('Shape of test_data tensor:', test_data.shape)

In [17]:
WV_DIM = 100
nb_words = min(MAX_NB_WORDS, len(word_vectors.vocab))
# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(nb_words, WV_DIM) - 0.5) / 5.0
for word, i in vocab.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass 
print(wv_matrix.shape)
wv_matrix
length=200

(6769, 100)


In [18]:
import keras.engine.input_layer as ki
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
def define_model(length,vocab_size):
  # channel 1
  inputs1 = ki.Input(shape=(length,))
  embedding1 = Embedding(nb_words,WV_DIM,mask_zero=False,weights=[wv_matrix],input_length=MAX_SEQUENCE_LENGTH)(inputs1)
  conv1 = Conv1D(filters=32, kernel_size=4, activation='tanh')(embedding1)
  drop1 = Dropout(0.3)(conv1)
  pool1 = MaxPooling1D(pool_size=2)(drop1)
  conv2 = Conv1D(filters=64, kernel_size=3, activation='tanh')(pool1)
  drop2 = Dropout(0.3)(conv2)
  pool2 = MaxPooling1D(pool_size=2)(drop2)
  flat1 = Flatten()(pool2)
  # merge
  merged = flat1
  # interpretation
  dense1 = Dense(10, activation='tanh')(merged)
  outputs = Dense(1, activation='sigmoid')(dense1)
  model = Model(inputs=inputs1, outputs=outputs)
  # compile
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize
  #print(model.summary())
  return model
model=define_model(length,vocab_size)
model.fit([train_x],np.array(train_data['label']),epochs=40,batch_size=128,validation_split=0.1)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 8668 samples, validate on 964 samples
Epoch 1/40
8668/8668 [==============================] - 8s 958us/step - loss: 0.3587 - acc: 0.8862 - val_loss: 0.3322 - val_acc: 0.8900
Epoch 2/40
8668/8668 [==============================] - 0s 54us/step - loss: 0.3401 - acc: 0.8873 - val_loss: 0.3269 - val_acc: 0.8900
Epoch 3/40
8668/8668 [==============================] - 0s 52us/step - loss: 0.3219 - acc: 0.8875 - val_loss: 0.3043 - val_acc: 0.8900
Epoch 4/40
8668/8668 [==============================] - 0s 53us/step - loss: 0.2433 - acc: 0.9099 - val_loss: 0.2908 - val_acc: 0.8932
Epoch 5/40
8668/8668 [==============================] - 0s 52us/step - loss: 0.1708 - acc: 0.9407 - val_loss: 0.3350 - val_acc: 0.8849
Epoch 6/40
8668/8668 [==============================]